# slack投稿メッセージをユーザーごとにグルーピング

## 処理の流れ概要

1. load user info, and make table (username, userid), and save it.
2. load message info , and make table(userid, msg_array), and save it.

## 1. load user/message/channels

In [1]:
import json
input_root = '../../data/010_raw'
output_root = '../../data/020_intermediate'
with open(input_root + '/' + 'user_info.json', 'r', encoding='utf-8') as f:
    user_info_rawdict = json.load(f)
with open(input_root + '/' + 'messages_info.json', 'r', encoding='utf-8') as f:
    msgs_info_rawdict = json.load(f)
with open(input_root + '/' + 'channel_info.json', 'r', encoding='utf-8') as f:
    ch_info_rawdict = json.load(f)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 1864: invalid start byte

In [2]:
user_info_rawdict

NameError: name 'user_info_rawdict' is not defined

In [29]:
msgs_info_rawdict[0]

{'channel_id': 'CJCN9CH43',
 'messages': [{'client_msg_id': 'a8f42e19-508d-431c-886f-061b45dda47c',
   'type': 'message',
   'text': '<https://public.tableau.com/profile/covid.19.data.resource.hub#!/vizhome/COVID-19Cases_15840488375320/COVID-19Cases>',
   'user': 'ULKPET3UK',
   'ts': '1584893215.279700',
   'team': 'TJGQW51Q9',
   'attachments': [{'title': 'COVID-19 Data Resource Hub - Profile | Tableau Public',
     'title_link': 'https://public.tableau.com/profile/covid.19.data.resource.hub#!/vizhome/COVID-19Cases_15840488375320/COVID-19Cases',
     'text': 'COVID-19 Data Resource Hub’s Tableau Public author profile page. See interactive data visualizations published by this author.',
     'fallback': 'COVID-19 Data Resource Hub - Profile | Tableau Public',
     'thumb_url': 'https://public.tableau.com/avatar/42cbf934-b491-4d22-85dc-ce3e8b93aa71.jpeg',
     'from_url': 'https://public.tableau.com/profile/covid.19.data.resource.hub#!/vizhome/COVID-19Cases_15840488375320/COVID-19Cases

In [30]:
ch_info_rawdict[0]

{'id': 'CJCN9CH43',
 'name': '04-1-ニュース-話題',
 'is_channel': True,
 'created': 1557922672,
 'is_archived': False,
 'is_general': False,
 'unlinked': 0,
 'creator': 'UJRAL005U',
 'name_normalized': '04-1-ﾆｭｰｽ-話題',
 'is_shared': False,
 'is_org_shared': False,
 'is_member': True,
 'is_private': False,
 'is_mpim': False,
 'members': ['UTR5X19NX',
  'UN0HCK74M',
  'USHRYKMED',
  'URXPGQ7U3',
  'URG3NEHGW',
  'UPR6BE9S6',
  'UT5FYCM6D',
  'UPF4T5DE1',
  'UR2D24RLK',
  'UP5KY3DFD',
  'URVQKNPDH',
  'UMBPQ87PB',
  'UT7T88ARH',
  'UR7A5CJNA',
  'URT07K7EE',
  'UT41LLNSV',
  'UT7E39YDR',
  'URDST28FJ',
  'UMC518ER0',
  'UL08YKGLU',
  'UUAUDBR1N',
  'URLV35G8M',
  'URDDG8EVA',
  'UKY4VFB5E',
  'UKG6BDAPL',
  'UQL12TR3M',
  'UTMLHKKK9',
  'URDDX224S'],
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': '', 'creator': '', 'last_set': 0},
 'previous_names': ['04-1-ニュース_話題',
  '04_1_ニュース_話題',
  '04_2_ニュース',
  '04_ニュース',
  '03_ニュース_話題のトピック',
  '03_1_ニュース_話題のトピック'],
 'num_memb

## make user table, message table

In [31]:
import pandas as pd
def make_user_table(usr_dict: dict) -> pd.DataFrame:
    uid_list = []
    uname_list = []
    for usr_ditem in usr_dict:
        if usr_ditem['deleted'] == True:
            continue
        uid_list.append(usr_ditem['id'])
        uname_list.append(usr_ditem['profile']['real_name_normalized'])
    user_table = pd.DataFrame({'uid': uid_list, 'uname': uname_list})
    return user_table
df_user_info = make_user_table(user_info_rawdict)
df_user_info

,uid,uname
0,USLACKBOT,Slackbot
1,UJKFAPBCJ,岡村龍弥
2,UJRAL005U,村上 智之
3,UJRDMQSAD,勝又健太 kenta.katsumata
4,UKEMYD9R8,tetsuroito
...,...,...
160,UV1GRG3UZ,Kazuhii
161,UV1J8NN6B,funakoshi
162,UV37036U8,Yuna
163,UV92NJEJH,奥津 勝太


In [32]:
# save user table
df_user_info.to_csv(output_root + '/' + 'users.csv', index=False)

In [33]:
def make_msg_table(msg_dict: dict) -> pd.DataFrame:
    ch_id_list = []
    msg_list = []
    uid_list = []
    ts_list = []
    for msg_ditem in msg_dict:
        if 'channel_id' in msg_ditem.keys():
            ch_id = msg_ditem['channel_id']
        else:
            continue
        if 'messages' in msg_ditem.keys():
            msgs_in_ch = msg_ditem['messages']
        else:
            continue
        # get message in channel
        for i, msg in enumerate(msgs_in_ch):
            # if msg by bot, continue
            if 'user' not in msg:
                continue
            ch_id_list.append(ch_id)
            msg_list.append(msg['text'])
            uid_list.append(msg['user']) # botの場合はこのキーがない
            ts_list.append(msg['ts'])
    df_msgs = pd.DataFrame({
        'ch_id': ch_id_list,
        'msg': msg_list,
        'uid': uid_list,
        'timestamp': ts_list
    })
    return df_msgs
df_msg_info = make_msg_table(msgs_info_rawdict)
df_msg_info

,ch_id,msg,uid,timestamp
0,CJCN9CH43,<https://public.tableau.com/profile/covid.19.d...,ULKPET3UK,1584893215.279700
1,CJCN9CH43,新型コロナの状況がTableauダッシュボードになった！\nTableauの中にあるデータを...,ULKPET3UK,1584893143.279500
2,CJCN9CH43,500€＋使った分ww\nたかいですねー、まあ翻訳にかなり力注いでるみたいですし、仕方ないっ...,URDDX224S,1584793581.277200
3,CJCN9CH43,初めてしりました！API使ってみたいですが、高い、、笑,UQGMHCCJ2,1584793196.275600
4,CJCN9CH43,google翻訳よりも良さげですね\n\n<https://www.deepl.com/tr...,URDDX224S,1584792107.273800
...,...,...,...,...
3467,CV1AUUVL4,<@UJRAL005U>さんがそのチャンネルをアーカイブしました,UJRAL005U,1583667206.007500
3468,CV1AUUVL4,大一発目はコロナネタです！\n\n■ 主題\nコロナによる各意思決定者ごとの思惑と最適戦略っ...,UJRAL005U,1583666515.007400
3469,CV1AUUVL4,<@ULJ74QZJM>さんがチャンネルに参加しました,ULJ74QZJM,1583665551.003900
3470,CV1AUUVL4,取り急ぎガイドラインとしては、\n\n・議論の主題を明確にする\n\n・その議論から得られる...,UJRAL005U,1583665392.003700


In [34]:
# save user table
df_msg_info.to_csv(output_root + '/' + 'messages.csv', index=False)